## Adaptation du tutoriel vers le texte

In [ ]:
import numpy as np
from math import floor
from keras.models import Sequential
from keras import layers
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
from random import randrange

from keras.utils import np_utils

from modules.loader import ComptaDataLoader

print('env: ', os.environ['CONDA_DEFAULT_ENV'])
%load_ext autoreload
%autoreload 2


## Téléchargement des données

In [ ]:
loader = ComptaDataLoader()
loader.ensure_data_loaded()

## Creation d'un CSV

In [ ]:
loader = ComptaDataLoader()
loader.create_csv(100)

## Lecture des csv

In [ ]:
path = r"../datas/CURATED/"

data = np.genfromtxt(path + '100_data.csv', delimiter=',')
labels = np.genfromtxt(path + '100_labels.csv', delimiter=',')


## Preprocessing d'image

In [ ]:
# Explication:
# 0     127.5   255   3 valeurs par defaut
# 0     .25     1     /255
# -.5   0       .5    -.5
data = (data / 255) - 0.5

data = np.apply_along_axis(lambda row: row.reshape(28,28), 1, data)
data = np.expand_dims(data, axis=3)

labels = to_categorical(labels)

# Mélange de manière synchronisé la data et les labels
randomize = np.arange(len(data))
np.random.shuffle(randomize)
data = data[randomize]
labels = labels[randomize]

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(data)


print(
    data.shape, '\n',
    labels.shape, '\n',
    [np.argmax(row) for row in labels[:10]]
)

## Je plot une lettre:

In [ ]:
nb_random = randrange(0, len(data))

img = data[nb_random]
plt.imshow(img, cmap='Greys')
print( "C'est un " + loader.int_to_letter(np.argmax(labels[nb_random])))

del(img, nb_random)

## Construction du modèle

In [ ]:
model = Sequential([
  layers.Conv2D(28, 3, activation='relu'),
  layers.Conv2D(56, 3, activation='relu'),
  layers.MaxPooling2D(2, 2),
  layers.Dropout(0.25),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.5),
  layers.Dense(26, activation='softmax')
])

model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

## Entrainement du modèle

In [ ]:
history = model.fit(
  data,
  labels,
  epochs=20,
  validation_split=0.25,
  verbose=2
)

model.save(str(int(len(data)/26)) + '_CNN.h5')

In [ ]:
def transform_to_letter(index: int):
    alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    return alphabet[index]

# from keras.models import load_model
# model = load_model('1000_CNN.h5')

nb_element = 29
nb_random = randrange(0, len(data)-nb_element)


prediction = [ transform_to_letter(lettre) for lettre in np.argmax(model.predict(data[nb_random:nb_random+nb_element]) , axis=1)]
verite = [ transform_to_letter(np.argmax(lettre))  for lettre in labels[nb_random:nb_random+nb_element] ] 

print(
    'Prediciton:\t', prediction,
    '\nVérité:\t\t', verite,
    '\nLe modèle donne un resultat juste ? ', prediction == verite
)

# Les hyperparamètres à tester:

## Network Depth
What happens if we add or remove Convolutional layers? How does that affect training and/or the model’s final performance?
```
model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),

  Conv2D(num_filters, filter_size),
  
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])
```

## Dropout
What if we tried adding Dropout layers, which are commonly used to prevent overfitting (surentrainement) ?
```
from tensorflow.keras.layers import Dropout

model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  
  Dropout(0.5),

  Flatten(),
  Dense(10, activation='softmax'),
])
```

## Fully-connected Layers
What if we add fully-connected layers between the Convolutional outputs and the final Softmax layer? This is something commonly done in CNNs used for Computer Vision.

```
model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),

  Dense(64, activation='relu'),
  
  Dense(10, activation='softmax'),
])
```

## Convolution Parameters

What if we play with the Conv2D parameters? For example:

```
Conv2D(
    num_filters,
    filter_size,
    input_shape=(28, 28, 1),

    strides=2,
    padding='same',
    activation='relu',
    
  ),
```

code complet du tutoriel :

``` python
# The full CNN code!
####################
import numpy as np
import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.utils import to_categorical

train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Reshape the images.
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

num_filters = 8
filter_size = 3
pool_size = 2

# Build the model.
model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])

# Compile the model.
model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# Train the model.
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=3,
  validation_data=(test_images, to_categorical(test_labels)),
)

# Save the model to disk.
model.save_weights('cnn.h5')

# Load the model from disk later using:
# model.load_weights('cnn.h5')

# Predict on the first 5 test images.
predictions = model.predict(test_images[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]
```